In [1]:
# Basic library
import nltk
import logging

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.util import align_tokens
from nltk.tokenize.api import TokenizerI
from nltk.tokenize import word_tokenize
from recordlinkage.index import Block
import recordlinkage as rl
import recordlinkage
import jellyfish
import warnings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
#date = ['curp']
bd = pd.read_csv(
    'dgis_datos/defuncion/seed2010_conv.txt', delimiter='|',
    parse_dates=False,
    encoding='latin-1')
cols = ['edad_c', 'edad', 'nom_c', 'nac_dia', 'nac_mes', 'nac_anio', 'sexo', 'afiliacion', 'curp', 'edo_civil', 'escolar', 'ocupacion']
certific = ['cert_por', 'cert_ced', 'cert_nom', 'num_acta', 'informante']
residencia = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_sitio', 'def_dia', 'def_mes', 'def_anio']

bd = bd[cols + residencia + certific]
#list(bd.loc[[10, 12]]['FECINISI'])

C:\Users\Hapy_\AppData\Local\Temp\ipykernel_8036\3039953664.py:2: DtypeWarning: Columns (32,33,38,39,44,76,107,108,116) have mixed types. Specify dtype option on import or set low_memory=False.
  bd = pd.read_csv(


In [53]:
##########################Campos vacios##########################
print('Total de regs: ', len(bd))
vacios = [[]]
for colum in cols:
    vacios[0].append(len(bd[pd.isna(bd[colum])]))

print(vacios)

##########################Edad_c##########################
#0: No especificado
#9: Se ignora
tipoEdadNoEp = len(bd[(bd.edad_c == 0) | (bd.edad_c == 9)])
print('Edad no especificada: ', tipoEdadNoEp)
vacios[0][0] += tipoEdadNoEp
##########################Edad##########################
#Edades de 0 a 130 años
#999: Desconocido 
edadNoEp = len(bd[(bd.edad == 999)])
print('Edad desconocida: ', edadNoEp)
vacios[0][1] += edadNoEp

##########################Nom_c##########################
#[Desconocido Desconocido Desconocido], [x x x], [Desconocida Desconocida Desconocida], [x x Desconocido Trece], [cuatro jur seis Desconocido]
#[S/A], [SN SN SN]
#vacios[0][2] += tipoEdadNoEp

##########################Nac_Dia##########################
#99: Desconocido
diaNac_NoEp = len(bd[(bd.nac_dia == 99)])
print('Dia_Nacimiento desconocido: ', diaNac_NoEp)
vacios[0][3] += diaNac_NoEp

##########################Nac_Mes##########################
#99: Desconocido
mesNac_NoEp = len(bd[(bd.nac_mes == 99)])
print('Mes_Nacimiento desconocido: ', mesNac_NoEp)
vacios[0][4] += mesNac_NoEp

##########################Nac_Anio##########################
#99: Desconocido
anioNac_NoEp = len(bd[(bd.nac_anio == 9999)])
print('Anio_Nacimiento desconocido: ', anioNac_NoEp)
vacios[0][5] += anioNac_NoEp

##########################Sexo##########################
#0: No especificado
#9: Se ignora
sexo_NoEp = len(bd[(bd.sexo == 0) | (bd.sexo == 9)])
print('Sexo desconocido: ', sexo_NoEp)
vacios[0][6] += sexo_NoEp

##########################Afiliacion##########################
#0,1,2,3,4,99
#vacios[0][7] += sexo_NoEp

##########################curp##########################
#vacios[0][8] += sexo_NoEp

##########################edo_civil (id_conyugal)##########################
#0: No especificado
#9: Se ignora
edoCiv_NoEp = len(bd[(bd.edo_civil == 0) | (bd.edo_civil == 9)])
print('Estado civ desconocido: ', edoCiv_NoEp)
vacios[0][9] += edoCiv_NoEp

##########################Escolar##########################
#0: No especificado
#99: Se ignora
escolar_NoEp = len(bd[(bd.escolar == 0) | (bd.escolar == 99)])
print('Escolaridad desconocida: ', escolar_NoEp)
vacios[0][10] += escolar_NoEp

##########################ocupacion##########################
#0: No especificado
#99: ocupaciones no especificadas
#999: Se ignora
Ocup_NoEp = len(bd[(bd.ocupacion == 0) | (bd.ocupacion == 99) | (bd.ocupacion == 999)])
print('Ocupaciones desconocidas: ', Ocup_NoEp)
vacios[0][11] += Ocup_NoEp


datavacios = pd.DataFrame(vacios, columns=cols)
datavacios

Total de regs:  560195
[[0, 0, 22689, 0, 0, 0, 0, 0, 513303, 0, 0, 35406]]
Edad no especificada:  486
Edad desconocida:  486
Dia_Nacimiento desconocido:  2739
Mes_Nacimiento desconocido:  2739
Anio_Nacimiento desconocido:  2745
Sexo desconocido:  236
Estado civ desconocido:  10129
Escolaridad desconocida:  25734
Ocupaciones desconocidas:  76


,edad_c,edad,nom_c,nac_dia,nac_mes,nac_anio,sexo,afiliacion,curp,edo_civil,escolar,ocupacion
0,486,486,22689,2739,2739,2745,236,0,513303,10129,25734,35482


In [139]:
#bd[(pd.isna(bd['nom_c']) == False) & (bd['nom_c'] != 'DESCONOCIDO DESCONOCIDO DESCONOCIDO') & (bd['nom_c'] != 'DESCONOCIDA DESCONOCIDA DESCONOCIDA')]
#(pd.isna(bd['nom_c']) == False)


sinNaN = bd[(pd.isna(bd['nom_c']) == False)]
sinDesconocido = sinNaN[(sinNaN['nom_c'].str.contains('DESCONOCIDO', case=False) == False) & (sinNaN['nom_c'] != 'DESCONOCIDO DESCONOCIDO DESCONOCIDO')]
sinDesconocida = sinDesconocido[(sinDesconocido['nom_c'].str.contains('DESCONOCIDA', case=False) == False) & (sinDesconocido['nom_c'] != 'DESCONOCIDA DESCONOCIDA DESCONOCIDA')]
#n
sinN = sinDesconocida[(sinDesconocida['nom_c'] != 'N N N')]
#x
sinX = sinN[(sinN['nom_c'].str.contains('^x X (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$)', case=False)) ==False]
#sn, sr
#SIN INFORMACION
#rn, r n, niña, niño (recien nacido)


########################################################################
#sinDesconocida[(sinDesconocida['nom_c'].str.contains('s/a', case=False))]
sinDesconocida[(sinDesconocida['nom_c'].str.contains('^N N (N$|FEMENINO$|MASCULINO$|RECIEN NACIDO$|RN$|R N$)', case=False))]

C:\Users\Hapy_\AppData\Local\Temp\ipykernel_8036\66716010.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinX = sinN[(sinN['nom_c'].str.contains('^x X (RECIEN NACIDO|RN|X|R N)', case=False)) ==False]
C:\Users\Hapy_\AppData\Local\Temp\ipykernel_8036\66716010.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinDesconocida[(sinDesconocida['nom_c'].str.contains('^N N (N$|FEMENINO|MASCULINO|RECIEN NACIDO|RN|R N$)', case=False))]


,edad_c,edad,nom_c,nac_dia,nac_mes,nac_anio,sexo,afiliacion,curp,edo_civil,...,def_mun,def_sitio,def_dia,def_mes,def_anio,cert_por,cert_ced,cert_nom,num_acta,informante
361,0,999,N N N,99,99,9999,1,99,NaN,9,...,6009,10,17,4,2010,2,22618,DR. MANUEL CABALLERO,174.0,NaN
6272,3,1,N N N,20,5,2010,1,1,NaN,1,...,20067,10,21,5,2010,2,1732046,MCL RODOLFO MENDOZA RMZ,349.0,NaN
21418,4,2,N N FEMENINO,3,12,2009,2,1,NaN,1,...,20067,11,3,2,2010,2,1692531,EPIGMENIO RAFAEL REYES CORTES,175.0,EPIGEMNIO RAFAEL REYES CORTES
49184,5,20,N N N,1,3,1990,1,99,NaN,1,...,20184,10,14,5,2010,2,4715617,DR. CHRISTIAN SOTO PALMA,160.0,NO HUBO
67831,5,28,N N N,1,3,1982,1,99,NaN,1,...,20184,10,2,9,2010,2,22610,DR. JAIME ABRAHAM DIAZ,374.0,NaN
80356,5,32,N N N,1,4,1978,1,99,NaN,1,...,20184,10,2,9,2010,2,22610,DR. JAIME ABRAHAM JIMENEZ DIAZ,373.0,NaN
89408,5,35,N N N,1,5,1975,1,1,NaN,1,...,20043,10,18,8,2010,2,4192682,DR. GABRIEL MARQUEZ PEÃA,370.0,AGENTE DEL MINISTERIO PUBLICO
160705,5,52,N N N,17,7,1958,1,1,NaN,5,...,12001,1,11,10,2010,3,3746212,ZARAGOZA ORTEGON DURAN,1938.0,NaN
210312,5,60,N N N,1,1,1950,1,99,NaN,9,...,20515,1,20,2,2010,2,4192682,DR. GABRIEL MARQUEZ PEÃA,83.0,AGENTE DEL MINISTERIO PUBLICO
210811,5,60,N N N,1,1,1950,1,0,NaN,1,...,6009,10,22,12,2010,2,1577438,JOSE FELIX GOMEZ,568.0,NaN
